# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
# Preview the data
df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Check Dtype value count for cleaning. Objects will be changed to floats to fit into the model
print(df.dtypes.value_counts())

float64    76
object     10
dtype: int64


In [6]:
# Identify which columns are objects
print(df.select_dtypes(include=['object']))

      home_ownership verification_status   issue_d loan_status pymnt_plan  \
0               RENT     Source Verified  Mar-2019    low_risk          n   
1           MORTGAGE            Verified  Mar-2019    low_risk          n   
2           MORTGAGE            Verified  Mar-2019    low_risk          n   
3               RENT            Verified  Mar-2019    low_risk          n   
4           MORTGAGE        Not Verified  Mar-2019    low_risk          n   
...              ...                 ...       ...         ...        ...   
68812           RENT     Source Verified  Jan-2019    low_risk          n   
68813           RENT        Not Verified  Jan-2019    low_risk          n   
68814       MORTGAGE     Source Verified  Jan-2019    low_risk          n   
68815       MORTGAGE            Verified  Jan-2019    low_risk          n   
68816       MORTGAGE            Verified  Jan-2019    low_risk          n   

      initial_list_status next_pymnt_d application_type hardship_flag  \
0 

In [7]:
# Use value_counts to dentify what needs to be changed

print(df['home_ownership'].value_counts())
print(df['verification_status'].value_counts())
print(df['loan_status'].value_counts())
print(df['pymnt_plan'].value_counts())
print(df['initial_list_status'].value_counts())
print(df['initial_list_status'].value_counts())
print(df['hardship_flag'].value_counts())
print(df['debt_settlement_flag'].value_counts())

MORTGAGE    36219
RENT        24613
OWN          7346
ANY           639
Name: home_ownership, dtype: int64
Not Verified       32895
Source Verified    25737
Verified           10185
Name: verification_status, dtype: int64
low_risk     68470
high_risk      347
Name: loan_status, dtype: int64
n    68817
Name: pymnt_plan, dtype: int64
w    60292
f     8525
Name: initial_list_status, dtype: int64
w    60292
f     8525
Name: initial_list_status, dtype: int64
N    68817
Name: hardship_flag, dtype: int64
N    68817
Name: debt_settlement_flag, dtype: int64


In [8]:
# Update the identified the object columns from objects to floats. Drop the dates as that information is not relevant.
df['home_ownership'] = df['home_ownership'].replace('OWN',0)
df['home_ownership'] = df['home_ownership'].replace('RENT',1)
df['home_ownership'] = df['home_ownership'].replace('MORTGAGE',2)
df['home_ownership'] = df['home_ownership'].replace('ANY',3)

df['verification_status'] = df['verification_status'].replace('Not Verified',0)
df['verification_status'] = df['verification_status'].replace('Source Verified',1)
df['verification_status'] = df['verification_status'].replace('Verified',2)

df['loan_status'] = df['loan_status'].replace("low_risk",0)
df['loan_status'] = df['loan_status'].replace("high_risk",1)

df['pymnt_plan'] = df['pymnt_plan'].replace("n",0)

df['initial_list_status'] = df['application_type'].replace("Individual",0)
df['initial_list_status'] = df['initial_list_status'].replace("Joint App",1)


df['application_type'] = df['application_type'].replace("Individual",0)
df['application_type'] = df['application_type'].replace("Joint App",1)


df['hardship_flag'] = df['hardship_flag'].replace("N",0)

df['debt_settlement_flag'] = df['debt_settlement_flag'].replace("N",0)

df.drop(["issue_d","next_pymnt_d"], axis=1, inplace=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,1,0,0,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,2,105000.0,2,0,0,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,2,56000.0,2,0,0,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,1,92000.0,2,0,0,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,2,52000.0,0,0,0,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [9]:
# Create our features
X = df.copy()
X.drop("loan_status",axis=1, inplace=True)
X.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,1,0,27.24,0.0,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,2,105000.0,2,0,20.23,0.0,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,2,56000.0,2,0,24.26,0.0,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,1,92000.0,2,0,31.44,0.0,1.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,2,52000.0,0,0,18.76,0.0,1.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [10]:
# Create our target
y = df['loan_status']
y.head()


0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [11]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.438133,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.691552,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,2.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,2.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [12]:
# Check the balance of our target values
print(y.value_counts())

0    68470
1      347
Name: loan_status, dtype: int64


In [13]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(51612, 83)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
brf_y_pred = brf.predict(X_test)
brf_bas = balanced_accuracy_score(y_test, brf_y_pred)
brf_bas

0.7198750335744519

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
brf_cm = confusion_matrix(y_test, brf_y_pred)
brf_cm

array([[14126,  2978],
       [   39,    62]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, brf_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.83      0.61      0.90      0.71      0.52     17104
          1       0.02      0.61      0.83      0.04      0.71      0.50       101

avg / total       0.99      0.82      0.62      0.90      0.71      0.52     17205



In [21]:
# List the features sorted in descending order by feature importance
brf_importances = brf.feature_importances_

brf_importances_sorted =sorted(zip(brf_importances, X.columns),reverse=True)

brf_importances_sorted

[(0.07833910832691689, 'last_pymnt_amnt'),
 (0.06030426537382076, 'total_rec_prncp'),
 (0.05692558620044764, 'total_pymnt_inv'),
 (0.05376103118807058, 'total_rec_int'),
 (0.05345411997401573, 'total_pymnt'),
 (0.029644357048149318, 'int_rate'),
 (0.019412656551714272, 'dti'),
 (0.018422420100730842, 'installment'),
 (0.018403957495210316, 'mths_since_recent_inq'),
 (0.018213441853042155, 'annual_inc'),
 (0.018203824988963634, 'out_prncp_inv'),
 (0.01711814092586331, 'il_util'),
 (0.016935937656538235, 'max_bal_bc'),
 (0.0167874590294067, 'total_bal_ex_mort'),
 (0.016735397075322168, 'total_rev_hi_lim'),
 (0.016714852291099945, 'revol_bal'),
 (0.015901453581351328, 'mo_sin_old_rev_tl_op'),
 (0.015469819050001927, 'tot_cur_bal'),
 (0.015232177389152092, 'out_prncp'),
 (0.01449942813394861, 'total_il_high_credit_limit'),
 (0.014430902875220144, 'tot_hi_cred_lim'),
 (0.01441971094907139, 'bc_util'),
 (0.01422593615827002, 'bc_open_to_buy'),
 (0.014107745964865494, 'mths_since_recent_bc'),

### Easy Ensemble Classifier

In [22]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier()
eec.fit(X_train,y_train)

EasyEnsembleClassifier()

In [23]:
# Calculated the balanced accuracy score
eec_y_pred = eec.predict(X_test)
eec_bas = balanced_accuracy_score(y_test, eec_y_pred)
eec_bas

0.7208420356769073

In [24]:
# Display the confusion matrix
eec_cm = confusion_matrix(y_test, eec_y_pred)
eec_cm

array([[13143,  3961],
       [   33,    68]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, eec_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.77      0.67      0.87      0.72      0.52     17104
          1       0.02      0.67      0.77      0.03      0.72      0.51       101

avg / total       0.99      0.77      0.67      0.86      0.72      0.52     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier had the best balanced accuracy score (.7356)

2. Which model had the best recall score?

    The Balanced Random Forest had the best recall score (0.82)

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier had the best geometric mean score (0.73)

4. What are the top three features?
    (0.07833910832691689, 'last_pymnt_amnt'),
    (0.06030426537382076, 'total_rec_prncp'),
    (0.05692558620044764, 'total_pymnt_inv'),
    